# Baseline in Colab

## Import

In [ ]:
!pip install -q transformers accelerate bitsandbytes

In [ ]:
!huggingface-cli login

In [ ]:
import os
import re
import ast
import pandas as pd

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [ ]:
from google.colab import drive

drive.mount("/content/drive", force_remount=False)


def join_path(*args):
    return os.path.join("/content/drive/MyDrive/강화학습", *args)

In [ ]:
INPUT_DATA_PATH = join_path("test.csv")
OUTPUT_DATA_PATH = join_path("submission.csv")
MODEL_PATH = join_path("llama3")

## Model Load

In [ ]:
model_name = "meta-llama/Llama-3.1-8B-Instruct"  # 본 대회는 반드시 Llama-3.1-8B-Instruct 모델만 사용해야 합니다.

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True,  # 원본 모델은 Colab(VRAM 16GB)에서 돌아가지 않습니다... GPU 자원 있으면 알려주세요.
    torch_dtype=torch.float16,
)
# 4bit + fp16으로 돌리니까 VRAM 7.2GB 정도 먹네요
model.eval()

In [ ]:
tokenizer.save_pretrained(MODEL_PATH)
model.save_pretrained(MODEL_PATH)

## Load Data

In [ ]:
data = pd.read_csv(INPUT_DATA_PATH, encoding="utf-8-sig")

## Inference

In [ ]:
def make_prompt(context, question, choices):
    choices = ast.literal_eval(choices)

    # 프롬프트를 수정하여 모델에 전달할 수 있습니다.
    # 예시 프롬프트
    return f"""질문에 대해서 다음 선택지 중 반드시 하나만 답하시오. 다른 선택지는 고려하지 마시오.

            질문 : {context} {question}
            선택지: {choices[0]} ,{choices[1]} ,{choices[2]}

            답변:"""


def extract_answer(text):
    # 대회 측에서 제시한 예시이고 당연히 다른 방법 써도 됩니다.
    raw_answer = text.split("답변:")[-1].strip()  # 프롬프트를 제외한 답변만 추출
    result = re.search(r"답변:\s*([^\n\r:]+)", text)  # 정규 표현식으로 답변 추출
    answer = result.group(1).strip() if result else None
    return raw_answer, answer

In [ ]:
def predict_answer(context, question, choices, max_new_tokens=16):
    prompt = make_prompt(context, question, choices)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.2,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id
        )
    result = tokenizer.decode(output[-1], skip_special_tokens=True)
    raw_answer, answer = extract_answer(result)

    return pd.Series({"raw_input": prompt, "raw_output": raw_answer, "answer": answer})

In [ ]:
check_point_dir = join_path("checkpoint")
os.makedirs(check_point_dir, exist_ok=True)

for i in range(len(data)):
    row = data.loc[i]
    result = predict_answer(row["context"], row["question"], row["choices"])

    data.at[i, "raw_input"] = result["raw_input"]
    data.at[i, "raw_output"] = result["raw_output"]
    data.at[i, "answer"] = result["answer"]

    # 5000개마다 중간 저장
    if i % 5000 == 0:
        print(f"✅ Processing {i}/{len(data)} — 중간 저장 중...")
        data[["ID", "raw_input", "raw_output", "answer"]].to_csv(
            join_path("checkpoint", f"submission_checkpoint_{str(i)}.csv"),
            index=False,
            encoding="utf-8-sig",
        )

## Submission

In [ ]:
submission = data[["ID", "raw_input", "raw_output", "answer"]]
submission.to_csv(OUTPUT_DATA_PATH, index=False, encoding="utf-8-sig")